### **Итоговое задание Альмира и Соколова Александра**  
### по Проекту 6. Car Price prediction 
####  Юнит 6. Основные алгоритмы машинного обучения. Часть II (отредактирован 09.09.2020)
---

### 1. Импорт библиотек

In [5]:
import pandas as pd
import requests, json
import time
import os
print(os.listdir('./data'))
PATH_to_file = './data/'

['all_car_brands_and_their_models_from_auto_ru.csv']


### 2. Импорт данных о марках и моделях

In [6]:
df_marks_and_models = pd.read_csv(PATH_to_file+'all_car_brands_and_their_models_from_auto_ru.csv', index_col=0)
print('Кол-во марок: ', df_marks_and_models.shape[0])
display(df_marks_and_models.head(2))

Кол-во марок:  262


,Models
Mark,
AC,"['378_GT', 'ACE', 'ACECA', 'COBRA']"
ACURA,"['CL', 'CSX', 'EL', 'ILX', 'INTEGRA', 'LEGEND'..."


### 2. Отбор марок для парсинга

In [7]:
# кол-во объявлений на сайте на 9.9.2020 
dict_of_num_cars_of_models ={'AC':2, 'Acura':94, 'Alfa Romeo':65, 'Alpina':10, 
                             'AMC':2, 'Ariel':1, 'Asia':1, 'Aston Martin':25, 
                             'Audi':4093, 'Austin Healey':1, 'Bajaj':3, 
                             'Bentley':104, 'BMW':7423, 'Borgward':1, 
                             'Brilliance':10, 'Buick':17, 'BYD':22, 
                             'Cadillac':321, 'Changan':100, 'Chery':593, 
                             'Chevrolet':3670, 'Chrysler':316, 'Citroen':1142, 
                             'Dacia':9, 'Daewoo':1024, 'Daihatsu':47, 
                             'Daimler':5, 'Datsun':212, 'Delage':1, 'Derways':2, 
                             'DKW':1, 'Dodge':413, 'DongFeng':25, 'Doninvest':2, 
                             'DS':3, 'DW Hower':8, 'Eagle':1, 'Excalibur':1, 
                             'FAW':66, 'Ferrari':29, 'Fiat':206, 'Ford':4812, 
                             'Foton':2, 'GAC':14, 'Geely':302, 'Genesis':54, 
                             'GMC':26, 'Great Wall':420, 'Hafei':6, 'Haima':12, 
                             'Haval':110, 'Hawtai':4, 'Heinkel':1, 'Honda':1179, 
                             'HuangHai':1, 'Hummer':70, 'Hyundai':5712, 
                             'Infiniti':1041, 'Innocenti':1, 'Iran Khodro':21, 
                             'Isuzu':43, 'JAC':8, 'Jaguar':308, 'Jeep':466, 'JMC':1, 
                             'Kia':5595, 'LADA (ВАЗ)':8935, 'Lamborghini':14, 
                             'Lancia':8, 'Land Rover':1616, 'Lexus':985, 
                             'Lifan':295, 'Lincoln':62, 'Lotus':1, 'Luxgen':9, 
                             'Mahindra':1, 'Maserati':54, 'Maybach':7, 
                             'Mazda':2311, 'McLaren':5, 'Mercedes-Benz':8005, 
                             'Mercury':16, 'Metrocab':2, 'MG':10, 'MINI':313, 
                             'Mitsubishi':3935, 'Mitsuoka':3, 'Nissan':5776, 
                             'Oldsmobile':10, 'Opel':3131, 'Packard':2, 
                             'Peugeot':1636, 'Plymouth':13, 'Pontiac':38, 
                             'Porsche':714, 'Proton':3, 'PUCH':1, 'Ravon':112, 
                             'Renault':2771, 'Rolls-Royce':55, 'Rover':69, 
                             'Saab':147, 'Saturn':9, 'Scion':9, 'SEAT':77, 
                             'Shanghai Maple':1, 'Skoda':2810, 'Smart':89, 
                             'SsangYong':718, 'Subaru':934, 'Suzuki':878, 
                             'Tatra':5, 'Tesla':43, 'Tianma':2, 'Toyota':4648, 
                             'Triumph':1, 'Vauxhall':1, 'Volkswagen':7109, 
                             'Volvo':2069, 'Vortex':91, 'Wartburg':1, 'Willys':2, 
                             'Zotye':15, 'ZX':4, 'ГАЗ':935, 'Гоночный болид':2, 
                             'ЗАЗ':231, 'ЗИЛ':2, 'ИЖ':129, 'ЛуАЗ':45, 
                             'Москвич':198, 'СМЗ':4, 'ТагАЗ':68, 'УАЗ':1107}

In [8]:
# уберем модели с кол-вом объявлений ниже 300 и повысим регистр индексов 
list_keys_to_delete = []
for mark in dict_of_num_cars_of_models.keys():
    if dict_of_num_cars_of_models[mark] < 300:
        list_keys_to_delete.append(mark)
    
for key in list_keys_to_delete: del dict_of_num_cars_of_models[key]

for mark in list(dict_of_num_cars_of_models.keys()):
    mark_up = mark.upper()
    dict_of_num_cars_of_models[mark_up]=dict_of_num_cars_of_models.pop(mark)

In [9]:
# посмотрим на список получившихся моделей
print(*dict_of_num_cars_of_models, sep=',')

AUDI,BMW,CADILLAC,CHERY,CHEVROLET,CHRYSLER,CITROEN,DAEWOO,DODGE,FORD,GEELY,GREAT WALL,HONDA,HYUNDAI,INFINITI,JAGUAR,JEEP,KIA,LADA (ВАЗ),LAND ROVER,LEXUS,MAZDA,MERCEDES-BENZ,MINI,MITSUBISHI,NISSAN,OPEL,PEUGEOT,PORSCHE,RENAULT,SKODA,SSANGYONG,SUBARU,SUZUKI,TOYOTA,VOLKSWAGEN,VOLVO,ГАЗ,УАЗ


In [10]:
# Удалим отечественные модели
for key in ['LADA (ВАЗ)','ГАЗ','УАЗ']: del dict_of_num_cars_of_models[key]

In [11]:
# проверим что названия моделей совпадаю с названиями каталога сайта auto.Ru
list_marks_df = list(df_marks_and_models.index)
list_marks_to_check = []
for mark in dict_of_num_cars_of_models.keys():
    if mark not in list_marks_df:
        list_marks_to_check.append(mark)
print(f'всего марок с кол-вом объявлений более 300:= {len(dict_of_num_cars_of_models)}')
print('список марок, которые нужно проверить :=', *list_marks_to_check)

всего марок с кол-вом объявлений более 300:= 36
список марок, которые нужно проверить := GREAT WALL LAND ROVER MERCEDES-BENZ SSANGYONG


In [12]:
dict_of_num_cars_of_models['GREAT WALL']

420

In [13]:
# исправим несоответствия
dict_of_num_cars_of_models['GREAT_WALL']=dict_of_num_cars_of_models.pop('GREAT WALL')
dict_of_num_cars_of_models['LAND_ROVER']=dict_of_num_cars_of_models.pop('LAND ROVER')
dict_of_num_cars_of_models['MERCEDES']=dict_of_num_cars_of_models.pop('MERCEDES-BENZ')
dict_of_num_cars_of_models['SSANG_YONG']=dict_of_num_cars_of_models.pop('SSANGYONG')

In [14]:
# еще раз проверим что названия моделей совпадаю с названиями каталога сайта auto.Ru
list_marks_df = list(df_marks_and_models.index)
list_marks_to_check = []
for mark in dict_of_num_cars_of_models.keys():
    if mark not in list_marks_df:
        list_marks_to_check.append(mark)
print(f'всего марок с кол-вом объявлений более 300 := {len(dict_of_num_cars_of_models)}')
print('список марок, которые нужно проверить :=', *list_marks_to_check)

всего марок с кол-вом объявлений более 300 := 36
список марок, которые нужно проверить :=


In [15]:
# запишем список релевантных марок для парсинга
list_of_relevant_marks = list(dict_of_num_cars_of_models.keys())

In [16]:
sum_of_ads = sum(dict_of_num_cars_of_models.values())
print(f"примерное кол-во строк после парсинга должно быть := {sum_of_ads}")

примерное кол-во строк после парсинга должно быть := 89189


### 3. Подготовка к парсингу

In [17]:
HEADERS = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    'Content-Length': '137',
    'content-type': 'application/json',
    'Cookie': '_csrf_token=1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24; autoru_sid=a%3Ag5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270%7C1580931467355.604800.8HnYnADZ6dSuzP1gctE0Fw.cd59AHgDSjoJxSYHCHfDUoj-f2orbR5pKj6U0ddu1G4; autoruuid=g5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270; suid=48a075680eac323f3f9ad5304157467a.bc50c5bde34519f174ccdba0bd791787; from_lifetime=1580933172327; from=yandex; X-Vertis-DC=myt; crookie=bp+bI7U7P7sm6q0mpUwAgWZrbzx3jePMKp8OPHqMwu9FdPseXCTs3bUqyAjp1fRRTDJ9Z5RZEdQLKToDLIpc7dWxb90=; cmtchd=MTU4MDkzMTQ3MjU0NQ==; yandexuid=1758388111580931457; bltsr=1; ',
    'Host': 'auto.ru',
    'origin': 'https://auto.ru/cars/all/',
    'Referer': 'https://auto.ru/cars/all/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
    'x-client-app-version': '202002.03.092255',
    'x-client-date': '1580933207763',
    'x-csrf-token': '1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24',
    'x-page-request-id': '60142cd4f0c0edf51f96fd0134c6f02a',
    'x-requested-with': 'fetch'
}

In [18]:
url = "https://auto.ru/-/ajax/desktop/listing/"

In [19]:
# посмотрим какие столбцы нам нужны 
test = pd.read_csv(PATH_to_file+'test.csv')
print('Кол-во столбцов теста:', test.shape[1])
print('Названия столбцов теста:', list(test.columns))

# вывод всех столбцов датафрейма
pd.set_option('display.max_columns', None)
test.head(2)

FileNotFoundError: [Errno 2] No such file or directory: './data/test.csv'

In [20]:
# сделаем тестовый вызов запроса для BMW модели M6 и выведем его на экран, чтобы по нему можно было сделать функцию для создания 23 необходимых столбцов
model = 'M6'
param = {
                'catalog_filter' : [{"mark": 'BMW', "model": model}],
                'section': "used",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "top_days":"600",
                "geo_radius":200,
                "geo_id":[213],
                'page': 1
            }

response = requests.post(url, json=param, headers = HEADERS)
data = response.json()
for elem in data['offers']:
    print(elem)
    break

{'availability': 'IN_STOCK', 'category': 'cars', 'color_hex': '0000CC', 'delivery_info': {}, 'description': 'Премиальная акустическая система объемного звука Bang & Olufsen с 16 динамиками\nАдаптивный круиз-контроль\nСигнализация схода с полосы движения\nПроекция на лобовое стекло\n\nАвтомобиль в идеальном внешнем и техническом состоянии.\nОбслуживание автомобиля исключительно у официального дилера, все ТО согласно регламенту.\nСЕМЬ регистрационных действий, ТРИ Владельца за все время эксплуатации.\n\n"Royal Motors Club"  предоставляет полный спектр услуг по продажам автомобилей с пробегом:\n- Выгодные условия кредитования, одобрение по двум документам.\n- Trade In – обмен любого Вашего авто на представленный в нашей компании это\n   быстро, выгодно, безопасно!\n-  Мы осуществили все необходимые проверки и даем гарантию юридической\n   чистоты всех наших автомобилей.\n - Оптимальные программы по авто-страхованию ( ОСАГО, КАСКО)\n\nАвтомобиль прошел комплексную диагностику и предпродажн

In [21]:
# строка для проверки содержания элемента запроса
elem['additional_info']['hidden']

False

In [22]:
def utils_clean_values_dict(d_dict):
    for elem in d_dict:
        d_dict[elem] = 0
    return

In [23]:
dict_columns = {'bodyType':0, 'brand':0, 'color':0, 'fuelType':0, 'modelDate':0, 
                'name':0, 'numberOfDoors':0, 'productionDate':0, 
                'vehicleConfiguration':0, 'vehicleTransmission':0, 
                'engineDisplacement':0, 'enginePower':0, 'description':0, 
                'mileage':0, 'Комплектация':0, 'Привод':0, 'Руль':0, 
                'Состояние':0, 'Владельцы':0, 'ПТС':0, 'Таможня':0, 
                'Владение':0, 'price':0}

model = 'M6'
# создаем функцию для обработки 23 необходимых столбцов
def from_elem_to_dict(d_elem, d_dict):
    utils_clean_values_dict(d_dict)
    
    try: d_dict['bodyType'] = d_elem['vehicle_info']['configuration']['human_name']
    except: d_dict['bodyType'] = None

    try: d_dict['brand'] = d_elem['vehicle_info']['mark_info']['code']
    except: d_dict['brand'] = None

    try: d_dict['color'] = d_elem['color_hex']
    except: d_dict['color'] = None

    try: d_dict['fuelType'] = d_elem['lk_summary'].split()[-1]
    except: d_dict['fuelType'] = None

    try: d_dict['modelDate'] = d_elem['vehicle_info']['super_gen']['year_from']
    except: d_dict['modelDate'] = None

    try: d_dict['name'] =  d_elem['vehicle_info']['tech_param']['human_name']
    except: d_dict['name'] = None

    try: d_dict['numberOfDoors'] = d_elem['vehicle_info']['configuration']['doors_count']
    except: d_dict['numberOfDoors'] = None

    try: d_dict['productionDate'] = d_elem['documents']['year']
    except: d_dict['productionDate'] = None

    try: d_dict['vehicleTransmission'] = d_elem['vehicle_info']['tech_param']['transmission']
    except: d_dict['vehicleTransmission'] = None

    try: d_dict['engineDisplacement'] = d_elem['vehicle_info']['tech_param']['human_name'].split()[0]
    except: d_dict['engineDisplacement'] = None

    try: d_dict['vehicleConfiguration'] = elem['vehicle_info']['configuration']['body_type']+' '+d_dict['vehicleTransmission']+' '+d_dict['engineDisplacement']
    except: d_dict['vehicleConfiguration'] = None

    try: d_dict['enginePower'] = d_elem['vehicle_info']['tech_param']['power']
    except: d_dict['enginePower'] = None

    try: d_dict['description'] = d_elem['description']
    except: d_dict['description'] = None

    try: d_dict['mileage'] = d_elem['state']['mileage']
    except: d_dict['mileage'] = None

    try: d_dict['Комплектация'] = d_elem['vehicle_info']['complectation']
    except: d_dict['Комплектация'] = None

    try: d_dict['Привод'] = d_elem['lk_summary'].split(', ')[-2]
    except: d_dict['Привод'] = None

    try: d_dict['Руль'] = d_elem['vehicle_info']['steering_wheel']
    except: d_dict['Руль'] = None

    # 'Состояние' не найдено
    d_dict['Состояние'] = None

    try: d_dict['Владельцы'] = d_elem['documents']['owners_number']
    except: d_dict['Владельцы'] = None

    try: d_dict['ПТС'] = d_elem['documents']['pts']
    except: d_dict['ПТС'] = None

    try: d_dict['Таможня'] = d_elem['documents']['custom_cleared']
    except: d_dict['Таможня'] = None

    try: d_dict['Владение'] =  d_elem['documents']['purchase_date']
    except: d_dict['Владение'] = None


    try: d_dict['price'] = d_elem['price_info']['RUR']
    except: d_dict['price'] = None

    # дата размещения объявления решил добавить 
    try: d_dict['start_date'] =  d_elem['additional_info']['hot_info']['start_time']
    except: d_dict['start_date'] = None

    # статус объявления 
    try: d_dict['hidden'] =  d_elemelem['additional_info']['hidden']
    except: d_dict['hidden'] = None

    # модель 
    d_dict['model'] =  model

    return

from_elem_to_dict(elem, dict_columns)

In [24]:
display(pd.DataFrame(dict_columns).head(2))
test[test['enginePower']=='560 N12'].head(2)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,...,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model
available_options,Купе,BMW,0000CC,бензин,2012,4.4 AMT (560 л.с.),2,2012,COUPE ROBOT 4.4,ROBOT,...,LEFT,None,3,DUPLICATE,True,None,2500000,2020-09-20T17:03:09Z,None,M6
id,Купе,BMW,0000CC,бензин,2012,4.4 AMT (560 л.с.),2,2012,COUPE ROBOT 4.4,ROBOT,...,LEFT,None,3,DUPLICATE,True,None,2500000,2020-09-20T17:03:09Z,None,M6


NameError: name 'test' is not defined

### 4. Парсинг

In [25]:
auto_ru = {}

iter_of_ads, iter_of_marks, old_iter_of_ads, sum_ads, sum_ads_from_dict = 0,0,0,0,0
space = ' '
start_time_global = time.clock()

for mark in list_of_relevant_marks:
    start_time = time.clock()
    list_of_models = json.loads(df_marks_and_models.loc[mark, 'Models'].replace("'", '"'))
    for model in list_of_models:
        for num_page in range(1, 200):
            param = {
                'catalog_filter' : [{"mark": mark, "model": model}],
                'section': "all",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "top_days":"900",
                "geo_radius":200,
                "geo_id":[213],
                'page': num_page
            }
            response = requests.post(url, json=param, headers = HEADERS)
            data = response.json()

            if data['offers'] != []:
                for elem in data['offers']:
                    from_elem_to_dict(elem, dict_columns)
                    auto_ru[iter_of_ads] = dict_columns.copy()
                    iter_of_ads += 1
            else:
                break
    iter_of_marks +=1
    temp_1 = iter_of_ads-old_iter_of_ads
    temp_2 = dict_of_num_cars_of_models[mark]
    temp_3 = temp_1/temp_2
    temp_4 = space*abs(13-len(mark)-len(str(iter_of_marks)))
    temp_5 = int(time.clock() - start_time)
    temp_8 = round(temp_3*100,2)
    temp_7 = space*abs(18-len(str(temp_1))-len(str(temp_2))-len(str(temp_8))-len(str(temp_5)))
    temp_9 = int(time.clock() - start_time_global)
    

    sum_ads += temp_1
    sum_ads_from_dict += temp_2
    temp_def = sum_ads/sum_ads_from_dict
    temp_6 = (sum_of_ads*temp_def - iter_of_ads)*temp_9/sum_ads


    print (f'{iter_of_marks}.{mark} {temp_4} ушло:{temp_5} сек., стр. {temp_1} из {temp_2} {temp_8}%,{temp_7}всего стр. {iter_of_ads}, прогноз {int(temp_6/60)} мин.')

    old_iter_of_ads=iter_of_ads

print (f'Всего спарсили := {iter_of_ads} строк, на это ушло: {int((time.clock() - start_time_global)/60)} мин., ')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


1.AUDI          ушло:66 сек., стр. 3441 из 4093 84.07%,   всего стр. 3441, прогноз 22 мин.
2.BMW           ушло:107 сек., стр. 5867 из 7423 79.04%,  всего стр. 9308, прогноз 19 мин.
3.CADILLAC      ушло:13 сек., стр. 400 из 321 124.61%,    всего стр. 9708, прогноз 20 мин.
4.CHERY         ушло:11 сек., стр. 395 из 593 66.61%,     всего стр. 10103, прогноз 20 мин.
5.CHEVROLET     ушло:58 сек., стр. 2628 из 3670 71.61%,   всего стр. 12731, прогноз 19 мин.
6.CHRYSLER      ушло:13 сек., стр. 254 из 316 80.38%,     всего стр. 12985, прогноз 20 мин.
7.CITROEN       ушло:26 сек., стр. 913 из 1142 79.95%,    всего стр. 13898, прогноз 20 мин.
8.DAEWOO        ушло:15 сек., стр. 678 из 1024 66.21%,    всего стр. 14576, прогноз 19 мин.
9.DODGE         ушло:16 сек., стр. 308 из 413 74.58%,     всего стр. 14884, прогноз 20 мин.
10.FORD         ушло:65 сек., стр. 3342 из 4812 69.45%,   всего стр. 18226, прогноз 18 мин.
11.GEELY        ушло:6 сек., стр. 192 из 302 63.58%,      всего стр. 18418, прогноз

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [26]:
# запись датасета
df_auto_ru = pd.DataFrame(auto_ru).T
df_auto_ru.to_csv(PATH_to_file+'all_auto_ru_09_09_2020.csv', index=False)
print(f'кол-во строк датасета:= {len(df_auto_ru)}')

кол-во строк датасета:= 71336


In [27]:
# визуальный контроль
df_auto_ru.head(3)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,...,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model
0,Седан,AUDI,CACECB,бензин,1982,1.8 MT (90 л.с.),4,1984,SEDAN MECHANICAL 1.8,MECHANICAL,...,LEFT,None,2,ORIGINAL,True,None,100000,2020-11-29T13:46:39Z,None,100
1,Седан,AUDI,0000CC,бензин,1988,2.0 MT (115 л.с.),4,1989,SEDAN MECHANICAL 2.0,MECHANICAL,...,LEFT,None,3,DUPLICATE,True,None,90000,2020-11-15T10:23:16Z,None,100
2,Седан,AUDI,FAFBFB,бензин,1982,2.0 MT (112 л.с.),4,1987,SEDAN MECHANICAL 2.0,MECHANICAL,...,LEFT,None,2,ORIGINAL,True,None,55000,2020-11-28T21:17:36Z,None,100


Первоначальный парсинг показал, что на сайте много старых объявлений . Так как изначально за последние 90 дней мы получили 43% от всех объявлений по BMW, поэтому пришлось скачать за последние 900 дней, чтобы получить полные цифры заявленные на сайте. 

In [28]:
df_auto_ru.columns()

TypeError: 'Index' object is not callable